<a href="https://colab.research.google.com/github/Shantanu88-code/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad.net/grap

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-14 00:58:38--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.4’

postgresql-42.2.16. 100%[===================>] 979.38K  5.57MB/s    in 0.2s    

2021-12-14 00:58:38 (5.57 MB/s) - ‘postgresql-42.2.16.jar.4’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [5]:
df.drop_duplicates()

DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: string, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

### Create DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [7]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id" : "count"})
customers_df.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   35669025|                 1|
|   48198949|                30|
|   43355824|                 6|
|   33014261|                 6|
|   23493243|                 1|
|   30717305|                 4|
|   15714077|                 2|
|    7854719|                 1|
|   12761428|                 2|
|   14127895|                 1|
|   51451778|                 4|
|    3452965|                 1|
|   40430762|                 2|
|   27314089|                15|
|   11405991|                 1|
|   20368048|                 2|
|   33967841|                 2|
|   10418855|                 1|
|   43783459|                 1|
|   50843047|                 4|
+-----------+------------------+
only showing top 20 rows



In [8]:
customers_table_df = customers_df.withColumnRenamed("count(customer_id)","customer_count")
customers_table_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   35669025|             1|
|   48198949|            30|
|   43355824|             6|
|   33014261|             6|
|   23493243|             1|
|   30717305|             4|
|   15714077|             2|
|    7854719|             1|
|   12761428|             2|
|   14127895|             1|
|   51451778|             4|
|    3452965|             1|
|   40430762|             2|
|   27314089|            15|
|   11405991|             1|
|   20368048|             2|
|   33967841|             2|
|   10418855|             1|
|   43783459|             1|
|   50843047|             4|
+-----------+--------------+
only showing top 20 rows



In [9]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"])
products_df.drop_duplicates()


DataFrame[product_id: string, product_title: string]

In [10]:
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B012P7UPSM|Chicago Blackhawk...|
|B001GQ3VHG|Copag Poker Size ...|
|B008VS8M58|Baoer 223 5.56x45...|
|B005F06F4U|All Terrain Tackl...|
|B010T4IE2C|Swim Cap - 3 Pack...|
|B004RKJGLS|adidas Men's Spor...|
|B005V3DCBU|Nike Men's Perfor...|
|B00MHT9WN8|Green Bay Packers...|
|B001CSIRQ8|Isokinetics Inc. ...|
|B001KZ3NOO|Aottop Quality El...|
|B00005RCQS|Everlast 2'x6' Fo...|
|B00FA7RWVI|OGIO Men's Shredd...|
|B000MMH2HU|Blue Sea Systems ...|
|B00WJOATDS|Zengi AR15 45 Deg...|
|B00BOM2JNY|Black Mountain Pr...|
|B00OD5GAFM|Trijicon VCOG 1-6...|
|B00CAHDC1K|Naruto Headband f...|
|B004NLHXLG|Under Armour Men'...|
|B00L7OANWI|Yes4All Deep Tiss...|
|B0085PPSIQ|Lansky PS-MED01 B...|
+----------+--------------------+
only showing top 20 rows



In [11]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1WBPB8MDCCN8F|   48945260|B012P7UPSM|     409940130| 2015-08-31|
|R32M0YEWV77XG8|    5782091|B001GQ3VHG|     657746679| 2015-08-31|
| RR8V7WR27NXJ5|   45813853|B008VS8M58|     962442336| 2015-08-31|
|R1MHO5V9Z932AY|    1593730|B005F06F4U|      74305227| 2015-08-31|
|R16PD71086BD2V|   29605511|B010T4IE2C|     787185588| 2015-08-31|
|R1Z8IFGWTRWXT6|   11112959|B004RKJGLS|      94127483| 2015-08-31|
|R3AUMSHAW73HWN|     108031|B005V3DCBU|     526977496| 2015-08-31|
|R2KWDWFOHGX6FL|   13981540|B00MHT9WN8|      26521265| 2015-08-31|
|R3H9543FWBWFBU|   37993909|B001CSIRQ8|     652431165| 2015-08-31|
| RUANXOQ9W3OU5|   26040213|B001KZ3NOO|     635861713| 2015-08-31|
|R31673RTGEZSW7|   34657602|B00005RCQS|      72099763| 2015-08-31|
|R22OQLFSH42RCM|   14346192|B00FA7RWVI|     757354022| 2015-08

In [12]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|                Y|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
| RUANXOQ9W3OU5|          5|            0|          0|   N|     

In [13]:
from pyspark.sql.types import IntegerType

In [14]:
# Comvert string tpe to integer type.
# vine_df.withColumn("star_rating", vine_df["star_rating"].cast(IntegerType()))


### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [15]:
# Configure settings for RDS
# mode = "append"
# jdbc_url="jdbc:postgresql://<endpoint>:5432/<database name>"
# config = {"user":"postgres", 
          # "password": "<password>", 
          # "driver":"org.postgresql.Driver"}

# Store environmental variable
from getpass import getpass

password = getpass("Your password here")

# Configure RDS settings
mode = "append"
jdbc_url="jdbc:postgresql://module16challenge.cnoxgmnbdfuk.ca-central-1.rds.amazonaws.com:5432/Amazon_Vine_Analysis_db"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Your password here··········


In [20]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [17]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [22]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_table_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [21]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)